In [1]:
import pyarrow.plasma as plasma

## Before we start

start the Plasma store by 

```sh
plasma_store -m 1000000000 -s /tmp/plasma
```

which allows the Plasma store to use up 1GB of memory, and sets the socket to `/tmp/plasma`. 

In [2]:
client = plasma.connect("/tmp/plasma")

## Play with Numpy

In [3]:
import numpy as np

In [4]:
data = np.random.randn(32, 512, 1024)

In [5]:
data.nbytes

134217728

In [6]:
import pyarrow as pa

In [7]:
tensor = pa.Tensor.from_numpy(data)

In [8]:
object_id = plasma.ObjectID(np.random.bytes(20))

In [9]:
data_size = pa.ipc.get_tensor_size(tensor)

In [10]:
data_size

134217984

In [11]:
buf = client.create(object_id, data_size)

In [12]:
stream = pa.FixedSizeBufferWriter(buf)

In [13]:
pa.ipc.write_tensor(tensor, stream)

134217984

In [14]:
client.seal(object_id)

In [17]:
[buf2] = client.get_buffers([object_id])

In [18]:
reader = pa.BufferReader(buf2)

In [19]:
tensor2 = pa.ipc.read_tensor(reader)

In [20]:
data2 = tensor2.to_numpy()

In [24]:
data2.nbytes

134217728

In [22]:
data[0, 0, :10]

array([-1.53396885,  0.61753747,  0.69881721, -0.11615524,  1.51866401,
       -0.04793302, -1.92279344, -0.75430095,  0.78556291, -0.984257  ])

In [23]:
data2[0, 0, :10]

array([-1.53396885,  0.61753747,  0.69881721, -0.11615524,  1.51866401,
       -0.04793302, -1.92279344, -0.75430095,  0.78556291, -0.984257  ])

## Test OOM

Only allow the Plasma store to use up to 10000 bytes this time. See how the Plasma handles the out-of-memory error.
```sh
plasma_store -m 10000 -s /tmp/plasma
```

In [66]:
client = plasma.connect("/tmp/plasma")

In [67]:
capacity = client.store_capacity()

In [68]:
capacity

10000

In [69]:
object_id = plasma.ObjectID(np.random.bytes(20))

In [70]:
buf = client.create(object_id, capacity + 1)

PlasmaStoreFull: object does not fit in the plasma store

In [71]:
buf = client.create(object_id, capacity)

In [72]:
object_id2 = plasma.ObjectID(np.random.bytes(20))

In [73]:
buf = client.create(object_id2, 1)

PlasmaStoreFull: object does not fit in the plasma store